This notebook has been tested with python 3.9.12. The libraries have also been tested with python 3.10.

In [7]:
%pylab inline
import pandas as pd
import libaarhusxyz
import numpy as np
import requests
import os

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/home/redhog/Projects/Emerald/experimental-pipeline-inversion/env/lib/python3.11/site-packages/IPython/core/magics/pylab.py:166: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


# Data download

Download data linked from https://doi.org/10.25740/vj301kx0276 and place it in the folder data/IncisedValley. If this fails in the future, go to the above URL and check if they've changed their page structure, you might still be able to find the new URL:s.

In [8]:
if not os.path.exists("data/IncisedValley"):
    os.makedirs("data/IncisedValley")

# Data preparation

As this data is not in the original SkyTEM XYZ format, but has been converted to a CSV (the header has been stripped) we need to convert it back to that format.

In [9]:
r = requests.get("https://stacks.stanford.edu/file/druid:vj301kx0276/aem_processed_data_foothill_central_valley.csv")
assert r.status_code == 200
with open("data/IncisedValley/aem_processed_data_foothill_central_valley.csv", "wb") as f:
    f.write(r.content)

In [10]:
r = requests.get("https://stacks.stanford.edu/file/druid:vj301kx0276/20201231_20023_IVF_SkyTEM304_SKB.gex")
assert r.status_code == 200
with open("data/IncisedValley/20201231_20023_IVF_SkyTEM304_SKB.gex", "wb") as f:
    f.write(r.content)

In [11]:
x = libaarhusxyz.XYZ()
x.flightlines = pd.read_csv("data/IncisedValley/aem_processed_data_foothill_central_valley.csv")
x.model_info["scalefactor"] = 1 # Unit in file is V/(Am**4), if data is in picoV, set to 1e-12 :)
# Our code expects tild to be measured as degrees from vertical of the transmitter loop normal.
# Input data is referenced to positive x, so we need to rotate it...
x.flightlines.TILT_X -= 270
x.flightlines.TILT_Y -= 270

x.normalize()

x.dump("data/IncisedValley/aem_processed_data_foothill_central_valley.xyz")

# Split flightlines

Split the dataset into individual flightline files, as it's often more practical to work with and allows us to e.g. do an inversion of a single flightline easily for demo purposes.

In [12]:
for lineno, line in x.split_by_line().items():
    line.dump("data/IncisedValley/aem_processed_data_foothill_central_valley.%s.xyz" % (lineno,))